In [9]:
import numpy as np
import cv2
import os


class MonoVideoOdometery(object):
    def __init__(self,
                img_file_path,
                pose_file_path,
                focal_length = 718.8560,
                pp = (607.1928, 185.2157),
                lk_params=dict(winSize  = (21,21), criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01)),
                detector=cv2.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)):
        '''
        Arguments:
            img_file_path {str} -- File path that leads to image sequences
            pose_file_path {str} -- File path that leads to true poses from image sequence

        Keyword Arguments:
            focal_length {float} -- Focal length of camera used in image sequence (default: {718.8560})
            pp {tuple} -- Principal point of camera in image sequence (default: {(607.1928, 185.2157)})
            lk_params {dict} -- Parameters for Lucas Kanade optical flow (default: {dict(winSize  = (21,21), criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))})
            detector {cv2.FeatureDetector} -- Most types of OpenCV feature detectors (default: {cv2.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)})

        Raises:
            ValueError -- Raised when file either file paths are not correct, or img_file_path is not configured correctly
        '''

        self.file_path = img_file_path
        self.detector = detector
        self.lk_params = lk_params
        self.focal = focal_length
        self.pp = pp
        self.R = np.zeros(shape=(3, 3))
        self.t = np.zeros(shape=(3, 3))
        self.id = 0
        self.n_features = 0

        # try:
        #     if not all([".png" in x for x in os.listdir(img_file_path)]):
        #         raise ValueError("img_file_path is not correct and does not exclusively png files")
        # except Exception as e:
        #     print(e)
        #     raise ValueError("The designated img_file_path does not exist, please check the path and try again")

        # try:
        #     with open(pose_file_path) as f:
        #         self.pose = f.readlines()
        # except Exception as e:
        #     print(e)
        #     raise ValueError("The pose_file_path is not valid or did not lead to a txt file")

        # self.process_frame()


    def hasNextFrame(self):
        """Used to determine whether there are remaining frames
           in the folder to process

        Returns:
            bool -- Boolean value denoting whether there are still
            frames in the folder to process
        """

        # return self.id < len(os.listdir(self.file_path))
        return True


    def detect(self, img):
        '''Used to detect features and parse into useable format


        Arguments:
            img {np.ndarray} -- Image for which to detect keypoints on

        Returns:
            np.array -- A sequence of points in (x, y) coordinate format
            denoting location of detected keypoint
        '''

        p0 = self.detector.detect(img)

        return np.array([x.pt for x in p0], dtype=np.float32).reshape(-1, 1, 2)

    def featureTracking(self):
        kp2, st, err = cv2.calcOpticalFlowPyrLK(self.old_frame, self.current_frame, self.p0, None, **lk_params)  # shape: [k,2] [k,1] [k,1]

        st = st.reshape(st.shape[0])
        kp1 = self.p0[st == 1]
        kp2 = kp2[st == 1]
        for i in range(len(kp1)):
            print(kp1[i], kp2[i])

        return kp1, kp2

    def visual_odometery(self):
        if self.n_features < 2000:
            self.p0 = self.detect(self.old_frame)

        # Отфильтровываем точки с хорошей корреспонденцией
        good_new, good_old = self.featureTracking()

        if len(good_new) < 5 or len(good_old) < 5:
            print("Недостаточно точек для вычисления Essential Matrix")
            return

        good_new = good_new.astype(np.float32)
        good_old = good_old.astype(np.float32)

        if good_new.size == 0 or good_old.size == 0:
            print("Нет соответствий между кадрами")
            return
        # Вычисляем матрицу Essential Matrix
        E, mask = cv2.findEssentialMat(good_new, good_old, focal=self.focal, pp=self.pp, method=cv2.RANSAC, prob=0.999, threshold=1.0)

        # Восстанавливаем относительное движение (R и t)
        _, R, t, mask = cv2.recoverPose(E, good_new, good_old, focal=self.focal, pp=self.pp)

        # Обновляем общую трансформацию
        self.t = self.t + self.R @ t
        self.R = R @ self.R

        # Обновляем предыдущий кадр и точки
        self.old_frame = frame.copy()
        self.p0 = good_new.reshape(-1, 1, 2)

        # Обновляем количество ключевых точек
        self.n_features = len(self.p0)


    def get_mono_coordinates(self):
        # We multiply by the diagonal matrix to fix our vector
        # onto same coordinate axis as true values
        diag = np.array(
            [
                [-1, 0, 0],
                [0, -1, 0],
                [0, 0, -1]
            ]
        )
        print("Координаты:", self.t)
        adj_coord = np.matmul(diag, self.t)

        return adj_coord.flatten()

# img_path = "./images"
img_path = ""
# pose_path = "./pose"
pose_path = ""
focal = 718.8560
pp = (607.1928, 185.2157)
# Parameters for lucas kanade optical flow
lk_params = dict(winSize=(21, 21),
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))

vo = MonoVideoOdometery(img_path, pose_path, focal, pp, lk_params)
vo.id = 2
traj = np.zeros(shape=(600, 800, 3))
flag = False

DRAW_TRAJ = False


# To capture a video, you need to create a VideoCapture object. Its argument can be either the device index or the name of a video file
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

frame = None
old_frame = None

while True:
    # Capture frame-by-frame
    if frame is not None:
        old_frame = frame.copy()
    ret, frame = cap.read()
    if old_frame is None:
        continue

    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # Our operations on the frame come here

    cv2.imshow('frame', frame)

    # Обработка изображения:
    vo.old_frame = old_frame.copy()
    vo.current_frame = frame.copy()
    vo.visual_odometery()
    vo.id += 1
    # Конец обраотки

    print(vo.get_mono_coordinates())

    mono_coord = vo.get_mono_coordinates()
    # print(mono_coord)
    # print("x: {}, y: {}, z: {}".format(*[str(pt) for pt in mono_coord]))
    # print("true_x: {}, true_y: {}, true_z: {}".format(*[str(pt) for pt in true_coord]))

    if DRAW_TRAJ:
        draw_x, draw_y, draw_z = [int(round(x)) for x in mono_coord]

        traj = cv2.circle(traj, (draw_x + 400, draw_z + 100), 1, list((0, 255, 0)), 4)

        cv2.putText(traj, 'Estimated Odometry Position:', (30, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        cv2.putText(traj, 'Green', (270, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        cv2.imshow('trajectory', traj)




    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

if DRAW_TRAJ:
    cv2.imwrite("./images/trajectory.png", traj)

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

Координаты: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
Координаты: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Координаты: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
Координаты: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Координаты: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
Координаты: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Координаты: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
Координаты: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Координаты: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
Координаты: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Координаты: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
Координаты: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Координаты: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
Координаты: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Координаты: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
Координаты: [[0. 0. 0.]
 [0. 0. 0

error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\five-point.cpp:759: error: (-215:Assertion failed) E.cols == 3 && E.rows == 3 in function 'cv::decomposeEssentialMat'
